In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def _load_data():
    pass

In [4]:
train = pd.read_csv("../data/Train.csv")
test = pd.read_csv("../data/test.csv")

/Users/codehi/Documents/software/Anaconda/anaconda/envs/IntroToTensorFlow/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
ls ../data

Data Dictionary.xlsx  do_not_open/          preds.xlxs
Test.zip              lin_pred.csv          rf_preds.csv
Train.csv*            line_preds.csv        sale_id.csv
Train.zip             lr_preds.csv          sales_id.csv
ada_preds.csv         median_benchmark.csv  test.csv
casestudy.py          newfile.py


In [8]:
print("training set: " + str(train.shape) + " "+str(len(train['SalesID'].unique()))+" SalesID") 
print("testing set: " + str(test.shape)  +" "+ str(len(test['SalesID'].unique()))+" SalesID" )
missing = train.isnull().any().any() or train.isnull().any().any()
print("There is no missing values in the datasets" if not missing else "There are missing values in the dataset")

training set: (401125, 53) 401125 SalesID
testing set: (11573, 52) 11573 SalesID
There are missing values in the dataset


In [6]:
test.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1222837,902859,1376,121,3,1000,0.0,NaN,1/5/2012 0:00,375L,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
1,1222839,1048320,36526,121,3,2006,4412.0,Medium,1/5/2012 0:00,TX300LC2,...,None or Unspecified,"12' 4""",None or Unspecified,Yes,Double,NaN,NaN,NaN,NaN,NaN
2,1222841,999308,4587,121,3,2000,10127.0,Medium,1/5/2012 0:00,270LC,...,None or Unspecified,"12' 4""",None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
3,1222843,1062425,1954,121,3,1000,4682.0,Low,1/5/2012 0:00,892DLC,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
4,1222845,1032841,4701,121,3,2002,8150.0,Medium,1/4/2012 0:00,544H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
